In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve

from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import BinaryAccuracy, AUC
from keras.regularizers import l1_l2

from tensorflow.keras.callbacks import EarlyStopping


from imblearn.over_sampling import RandomOverSampler

2022-10-02 08:36:53.355794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-02 08:36:53.643759: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-02 08:36:53.654231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-02 08:36:53.654256: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
data = pd.read_csv('Washington_State_HDMA-2016.csv')

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (23,24,25,28,29,30,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# if a loan originated there is no denial_reason, we just wanted to check that
data[data["action_taken_name"] == "Loan originated"][["denial_reason_name_1", "denial_reason_name_2", "denial_reason_name_3"]].value_counts()

Series([], dtype: int64)

In [ ]:
data[data["denial_reason_name_3"] != ""][["denial_reason_name_1", "denial_reason_name_2", "denial_reason_name_3"]].value_counts()

denial_reason_name_1                            denial_reason_name_2           denial_reason_name_3                          
Debt-to-income ratio                            Credit history                 Other                                             75
                                                                               Credit application incomplete                     55
Credit history                                  Credit application incomplete  Other                                             55
Debt-to-income ratio                            Credit history                 Insufficient cash (downpayment, closing costs)    46
                                                Collateral                     Other                                             32
                                                                                                                                 ..
Credit application incomplete                   Other                          Ins

In [ ]:
# remove the unhelpful rows
# - Application withdrawn by applicant
# - Application approved but not accepted
# - Preapproval request denied by financial institution
# - Preapproval request approved but not accepted

data_clean = data.drop(data[(data.action_taken_name == "Application withdrawn by applicant") | \
                            (data.action_taken_name == "Application approved but not accepted") | \
                            (data.action_taken_name == "Preapproval request denied by financial institution") | \
                            (data.action_taken_name == "Preapproval request approved but not accepted")].index)

# remove approved loans
data_clean = data_clean[data_clean["action_taken_name"] == "Loan originated"]

# remove unnecessary columns
data_clean = data_clean.drop(columns=["action_taken_name", \
                         "sequence_number", \
                         "respondent_id", \
                         "purchaser_type_name"])

# turn denial reasons into categorial variables
pd.get_dummies(data=data_clean, prefix=["denial_reason_name_1"], columns=["denial_reason_name_1"]).columns


Index(['tract_to_msamd_income', 'rate_spread', 'population',
       'minority_population', 'number_of_owner_occupied_units',
       'number_of_1_to_4_family_units', 'loan_amount_000s',
       'hud_median_family_income', 'applicant_income_000s', 'state_name',
       'state_abbr', 'property_type_name', 'preapproval_name',
       'owner_occupancy_name', 'msamd_name', 'loan_type_name',
       'loan_purpose_name', 'lien_status_name', 'hoepa_status_name',
       'edit_status_name', 'denial_reason_name_3', 'denial_reason_name_2',
       'county_name', 'co_applicant_sex_name', 'co_applicant_race_name_5',
       'co_applicant_race_name_4', 'co_applicant_race_name_3',
       'co_applicant_race_name_2', 'co_applicant_race_name_1',
       'co_applicant_ethnicity_name', 'census_tract_number', 'as_of_year',
       'application_date_indicator', 'applicant_sex_name',
       'applicant_race_name_5', 'applicant_race_name_4',
       'applicant_race_name_3', 'applicant_race_name_2',
       'applicant_race

In [ ]:

"""
# replace nan with 0 in rate_spread
data_clean.fillna(0, inplace=True)

# turn categorial into dummies
categorial_names = list(set(data_clean.columns) - set(data_clean._get_numeric_data().columns))
data_clean = pd.get_dummies(data=data_clean, columns=categorial_names, drop_first=True)
"""

In [ ]:
X = data_clean.drop('loan_approved', axis=1).values
y = data_clean['loan_approved'].values

oversample = RandomOverSampler(sampling_strategy='minority')
X, y = oversample.fit_resample(X, y)

scaler = MinMaxScaler()
X = scaler.fit_transform(X, y)

print(X.shape)
print(y.shape)


(527424, 146)
(527424,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
epochs = 100

model = Sequential()

model.add(
        Dense(32, activation='relu', input_shape=(X_train.shape[1],))
)

model.add(
        Dense(32, activation='relu',)
)


model.add(
        Dense(1, activation='sigmoid')
)

model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy', 'AUC']
)

early_stop = EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        verbose=1, 
        patience=25
)

history = model.fit(
        X_train, 
        y_train, 
        epochs=epochs, 
        batch_size=32768, 
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        use_multiprocessing=True
)

2022-10-02 08:33:06.043302: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-02 08:33:06.043336: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-02 08:33:06.043350: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-03c1f381-4bef-44c8-9167-00386ebceba9): /proc/driver/nvidia/version does not exist
2022-10-02 08:33:06.043564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Epoch 1/100
12/12 [==============================] - 4s 171ms/st

KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=03c1f381-4bef-44c8-9167-00386ebceba9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>